In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('/kaggle/input/demand-forecasting/train_0irEZ2H.csv')
test_df = pd.read_csv('/kaggle/input/demand-forecasting/test_nfaJ3J5.csv')

In [ ]:
train_df.head(4000)

In [ ]:
train_df.info()

In [ ]:
# train_df[['day', 'month', 'year']] = train_df['week'].str.split('/', expand=True)
# test_df[['day', 'month', 'year']] = test_df['week'].str.split('/', expand=True)
train_df.week = pd.to_datetime(train_df.week)
test_df.week = pd.to_datetime(test_df.week)
train_df['day'] = train_df.week.dt.day
train_df['month'] = train_df.week.dt.month
train_df['year'] = train_df.week.dt.year

test_df['day'] = test_df.week.dt.day
test_df['month'] = test_df.week.dt.month
test_df['year'] = test_df.week.dt.year

In [ ]:
train_df

In [ ]:
train_df.describe()

1. one value is missing in total price column looking at count 
2. units sold has minimum value 1 and maximum 2876 which seems like an outlier because mean is 51.67 which is closer to minimum value. Also 75 percentile value is also 62

In [ ]:
train_df[train_df.units_sold==2876]

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df = train_df.dropna(axis=0)

In [ ]:
test_df.isnull().sum()

In [ ]:
train_df.hist(figsize=(10,10))

In [ ]:
h = plt.hist(np.log(train_df.units_sold), bins=30)

In [ ]:
sns.barplot(data=train_df, x='sku_id',  y='units_sold')
ticks = plt.xticks(rotation=90)
plt.show()

In [ ]:
sns.barplot(data=train_df[train_df.year==2011], x='sku_id',  y='units_sold')
ticks = plt.xticks(rotation=90)
plt.show()
sns.barplot(data=train_df[train_df.year==2012], x='sku_id',  y='units_sold')
ticks = plt.xticks(rotation=90)
plt.show()
sns.barplot(data=train_df[train_df.year==2013], x='sku_id',  y='units_sold')
ticks = plt.xticks(rotation=90)
plt.show()

In [ ]:
train_df.pivot_table(index='sku_id')

In [ ]:
sku_id_sortedby_unitssold = train_df.groupby('sku_id').describe()['units_sold']['mean'].sort_values(ascending=False).index
sku_id_sortedby_unitssold = list(sku_id_sortedby_unitssold)
train_df['sku_id_sortedby_unitssold'] = train_df['sku_id']
train_df['sku_id_sortedby_unitssold'] = train_df['sku_id_sortedby_unitssold'].apply(lambda x: len(sku_id_sortedby_unitssold)-sku_id_sortedby_unitssold.index(x))

test_df['sku_id_sortedby_unitssold'] = test_df['sku_id']
test_df['sku_id_sortedby_unitssold'] = test_df['sku_id_sortedby_unitssold'].apply(lambda x: len(sku_id_sortedby_unitssold)-sku_id_sortedby_unitssold.index(x))

In [ ]:
avg_by_sku_id = train_df.pivot_table(index='sku_id')['units_sold'].sort_values(ascending=False)
train_df['avg_by_sku_id'] = train_df['sku_id'].copy()
train_df['avg_by_sku_id'] = train_df.avg_by_sku_id.apply(lambda x:avg_by_sku_id[avg_by_sku_id.index==x].values[0])

test_df['avg_by_sku_id'] = test_df['sku_id'].copy()
test_df['avg_by_sku_id'] = test_df.avg_by_sku_id.apply(lambda x:avg_by_sku_id[avg_by_sku_id.index==x].values[0])

In [ ]:
plt.subplots(figsize=(15, 6))
sns.barplot(data=train_df, x='store_id',  y='units_sold')
ticks = plt.xticks(rotation=90)
plt.show

In [ ]:
store_id_sortedby_unitssold = train_df.groupby('store_id').describe()['units_sold']['mean'].sort_values(ascending=False).index
store_id_sortedby_unitssold = list(store_id_sortedby_unitssold)
train_df['store_id_sortedby_unitssold'] = train_df['store_id']
train_df['store_id_sortedby_unitssold'] = train_df['store_id_sortedby_unitssold'].apply(lambda x: len(store_id_sortedby_unitssold)-store_id_sortedby_unitssold.index(x))

test_df['store_id_sortedby_unitssold'] = test_df['store_id']
test_df['store_id_sortedby_unitssold'] = test_df['store_id_sortedby_unitssold'].apply(lambda x: len(store_id_sortedby_unitssold)-store_id_sortedby_unitssold.index(x))

In [ ]:
avg_by_store_id = train_df.pivot_table(index='store_id')['units_sold'].sort_values(ascending=False)
train_df['avg_by_store_id'] = train_df['store_id'].copy()
train_df['avg_by_store_id'] = train_df.avg_by_store_id.apply(lambda x:avg_by_store_id[avg_by_store_id.index==x].values[0])

test_df['avg_by_store_id'] = test_df['store_id'].copy()
test_df['avg_by_store_id'] = test_df.avg_by_store_id.apply(lambda x:avg_by_store_id[avg_by_store_id.index==x].values[0])

In [ ]:
sns.scatterplot(data=train_df, x='total_price', y='units_sold')

Let us remove the outliers

In [ ]:
train_df = train_df.drop(train_df[train_df.units_sold>1500].index, axis=0)

In [ ]:
fig, ax = plt.subplots(1,2)
sns.scatterplot(data=train_df, x=np.log(train_df.total_price), y='units_sold', ax=ax[0])
sns.scatterplot(data=train_df, x='base_price', y='units_sold', ax=ax[1])

In [ ]:
# train_df['log_transformed_tp'] = np.log(train_df.total_price)
# test_df['log_transformed_tp'] = np.log(test_df.total_price)

In [ ]:
plt.subplots(figsize=(15,5))
sns.lineplot(data=train_df, x='week', y='units_sold')
t = plt.xticks(rotation=90)

In [ ]:
# fig, ax = plt.subplots(1,2, figsize=(15,4))
# sns.lineplot(data=train_df, x='day', y='units_sold', ax=ax[0])
# sns.barplot(data=train_df, x='day', y='units_sold', ax=ax[1])

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(15,4))
sns.lineplot(data=train_df, x='month', y='units_sold', ax=ax[0])
sns.barplot(data=train_df, x='month', y='units_sold', ax=ax[1])

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(15,4))
sns.lineplot(data=train_df, x='year', y='units_sold', ax=ax[0])
sns.barplot(data=train_df, x='year', y='units_sold', ax=ax[1])

In [ ]:
train_df.groupby(['year', 'month'])['units_sold'].mean().plot()

In [ ]:
train_df.columns

In [ ]:
sns.lineplot(data=train_df[train_df.year==2011], x='week', y='units_sold')
plt.show()
sns.lineplot(data=train_df[train_df.year==2012], x='week', y='units_sold')
plt.show()
sns.lineplot(data=train_df[train_df.year==2013], x='week', y='units_sold')
plt.show()

In [ ]:
sns.scatterplot(data=train_df[train_df.year==2011], x='month', y='units_sold')
plt.show()
sns.scatterplot(data=train_df[train_df.year==2012], x='month', y='units_sold')
plt.show()
sns.scatterplot(data=train_df[train_df.year==2013], x='month', y='units_sold')
plt.show()

In [ ]:
# group_by_day_df = train_df.groupby(["month", "day", "sku_id"]).describe()['units_sold']

In [ ]:
# train_df['group_by_day'] = -1
# for i in range(group_by_day_df['mean'].values.shape[0]):
#     tup = group_by_day_df.index[i]
#     m = group_by_day_df['mean'].values[i]
#     train_df.loc[(train_df.month==tup[0])  & (train_df.day==tup[1]) & (train_df.sku_id==tup[2]), 'group_by_day'] = m
    

# test_df['group_by_day'] = -1
# for i in range(group_by_day_df['mean'].values.shape[0]):
#     tup = group_by_day_df.index[i]
#     m = group_by_day_df['mean'].values[i]
#     test_df.loc[(test_df.month==tup[0])  & (test_df.day==tup[1]) & (test_df.sku_id==tup[2]), 'group_by_day'] = m
    

In [ ]:
train_df['diff_bp_tp'] = train_df['total_price'] - train_df['base_price']
train_df['div_bp_tp'] = train_df['total_price']/train_df['base_price']

test_df['diff_bp_tp'] = test_df['total_price'] - test_df['base_price']
test_df['div_bp_tp'] = test_df['total_price']/test_df['base_price']

In [ ]:
train_df.columns

In [ ]:
features = ['is_featured_sku', 'is_display_sku','diff_bp_tp' ,'div_bp_tp','sku_id_sortedby_unitssold', 'store_id_sortedby_unitssold', 'avg_by_sku_id', 'avg_by_store_id', 'day', 'month', 'year', 'units_sold']
corr = train_df[features].corr()
features = features[:-1]
corr

In [ ]:
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(train_df.loc[:, train_df.columns != 'units_sold'], train_df.units_sold, test_size=0.2)

In [ ]:
def RMSLE(actual, predicted):

    predicted = np.array([np.log(np.abs(x+1.0)) for x in predicted])  # doing np.abs for handling neg values  
    actual = np.array([np.log(np.abs(x+1.0)) for x in actual])
    log_err = actual-predicted
    
    return 1000*np.sqrt(np.mean(log_err**2))

def get_e(model, df, target):
    pred = model.predict(df)
#     pred = np.exp(pred)
    pred[pred<=0] = 52
    er = RMSLE(target, pred)
    print(er)
    
def get_preds(model, df):
    preds = model.predict(df)
#     preds = np.exp(preds)
    print("Pred<=0: ", len(preds[preds<=0]))
    preds[preds<=0] = 52
    return preds
    
def preds_to_csv(m, preds):
    pd.DataFrame({'record_ID':test_df.record_ID, 'units_sold':preds}).to_csv('datahack-demand-forecasting-'+m+'.csv', index=False)

In [ ]:
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()
# X = scaler.fit_transform(train_df[features])
# X = X.reshape((X.shape[0], 9, 1))
# test_set = scaler.transform(test_df[features])
# test_set = test_set.reshape((test_set.shape[0], 9, 1))
# y = train_df.units_sold

In [ ]:
# from keras.models import Sequential
# from keras.layers import LSTM, Dense
# from keras.metrics import MeanSquaredLogarithmicError

# msle = MeanSquaredLogarithmicError()
# model = Sequential()
# model.add(LSTM(32, activation='relu', input_shape=(9,1)))
# model.add(Dense(1))
# model.compile(optimizer='RMSprop', loss='mean_squared_error', metrics=[msle])

In [ ]:
# model.fit(X, y, epochs=50, batch_size=32)

In [ ]:
# print("Training:")
# lstm_train_preds = get_preds(model=model, df=X)
# get_e(model=model, df=X, target=y)

In [ ]:
# lstm_preds = get_preds(model=model, df=test_set)
# preds_to_csv('lstm', lstm_preds.reshape(lstm_preds.shape[0], ))

In [ ]:
# from catboost import CatBoostRegressor

# cbr = CatBoostRegressor()
# cbr.fit(train_df[features], train_df.units_sold)

# print("Training:")
# cbr_train_preds = get_preds(model=cbr, df=train_df[features])
# get_e(model=cbr, df=train_df[features], target=train_df.units_sold)

In [ ]:
# cbr_preds = get_preds(model=cbr, df=test_df[features])
# preds_to_csv('cbr', cbr_preds)

In [ ]:
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.model_selection import RepeatedKFold, cross_val_score, KFold, StratifiedKFold

In [ ]:
print("Training:")
lgb_train_preds = get_preds(model=lgb, df=train_df[features])
get_e(model=lgb, df=train_df[features], target=train_df.units_sold)

In [ ]:
def runLGB(Xtrain, ytrain, Xval, yval, Xtest = None):
    params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'l1',
    #'max_depth': 9, 
    'learning_rate': 0.1
    ,'verbose': 1
    , "min_data_in_leaf" : 10
    }

    n_estimators = 800
    early_stopping_rounds = 10

    d_train = lgb.Dataset(Xtrain.copy(), label=ytrain.copy())
    d_valid = lgb.Dataset(Xval.copy(), label=yval.copy())
    watchlist = [d_train, d_valid]

    model = lgb.train(params, d_train, n_estimators
                      , valid_sets = [d_train, d_valid]
                      , verbose_eval=n_estimators
                      , early_stopping_rounds=early_stopping_rounds)

    preds = model.predict(Xval, num_iteration=model.best_iteration)
    err = RMSLE(yval, preds)
    
    preds_test = model.predict(Xtest, num_iteration=model.best_iteration)
    return  preds, err, preds_test, model

In [ ]:
pred_val, err, pred_test,model = runLGB(train_X[features], train_y, val_X[features], val_y, test_df[features])

In [ ]:
lgb_preds = get_preds(model=model, df=test_df[features])
preds_to_csv('lgb', lgb_preds)

In [ ]:
a =model.feature_importance(importance_type='split')
feature = pd.DataFrame(model.feature_name(), columns = ['feature'])
feature['importance'] = a
feature = feature.sort_values(by = ['importance'], ascending = False)
feature.head(11)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr=RandomForestRegressor(n_estimators=300)
# rfr.fit(train_df[features], np.log(train_df.units_sold))
rfr.fit(train_df[features], train_df.units_sold)

print("Training:")
rfr_train_preds = get_preds(model=rfr, df=train_df[features])
get_e(model=rfr, df=train_df[features], target=train_df.units_sold)

In [ ]:
rfr_preds = get_preds(model=rfr, df=test_df[features])
preds_to_csv('rfr', rfr_preds)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr=RandomForestRegressor(n_estimators=300,max_depth=100,n_jobs=-1)
rfr2.fit(train_X[features], train_y)

print("Training:")
rfr_train_preds = get_preds(model=rfr2, df=train_X[features])
get_e(model=rfr, df=train_X[features], target=train_y)
print("Testing:")
rfr_test_preds = get_preds(model=rfr2, df=val_X[features])
get_e(model=rfr, df=val_X[features], target=val_y)

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(10,8))
sns.lineplot(data=val_X, x='week',y=val_y, ax=ax[0])
sns.lineplot(data=val_X, x='week', y=rfr_test_preds, color='red', ax=ax[1])
ax[1].set_ylabel("units_sold")

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(10,8))
sns.scatterplot(data=val_X, x='month',y=val_y, ax=ax[0])
sns.scatterplot(data=val_X, x='month', y=rfr_test_preds, color='red', ax=ax[1])
ax[1].set_ylabel("units_sold")

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr=RandomForestRegressor(n_estimators=200, max_depth=100,n_jobs=-1)
rfr.fit(train_df[features], train_df.units_sold)

print("RFR:")
get_e(rfr)
rfr_preds = get_preds(rfr)

In [ ]:
pred_to_csv('rfr', rfr_preds)

In [ ]:
# import gc

# del rfr
# del cbr
# del gbr
# del lr

# gc.collect()

In [ ]:
# from sklearn.model_selection import GridSearchCV
# # Create the parameter grid based on the results of random search 
# param_grid = {
#     'bootstrap': [True],
#     'max_depth': [80, 90, 100, 110],
#     'min_samples_leaf': [3, 4, 5],
#     'min_samples_split': [8, 10, 12],
#     'n_estimators': [100, 200, 300, 400]
# }
# # Create a based model
# rf = RandomForestRegressor()
# # Instantiate the grid search model
# grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
#                           cv = 3, n_jobs = -1, verbose = 2)


In [ ]:
# grid_search.fit(train_df[features], train_df.units_sold)

# print("GS RFR:")
# get_e(grid_search)
# gs_preds = get_preds(grid_search)

In [ ]:
# print("GS RFR:")
# get_e(grid_search)
# gs_preds = get_preds(grid_search)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
rfr.get_params()